# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117115e+02     1.522678e+00
 * time: 0.7460339069366455
     1     1.004530e+01     8.572593e-01
 * time: 1.9987897872924805
     2    -1.171744e+01     9.688505e-01
 * time: 2.670334815979004
     3    -3.439892e+01     6.916078e-01
 * time: 3.629138946533203
     4    -4.773660e+01     4.689177e-01
 * time: 4.591064929962158
     5    -5.719931e+01     1.871125e-01
 * time: 5.551507949829102
     6    -6.000431e+01     9.365459e-02
 * time: 6.189083814620972
     7    -6.102534e+01     4.870668e-02
 * time: 6.835834980010986
     8    -6.141758e+01     5.989802e-02
 * time: 7.486119985580444
     9    -6.169720e+01     3.405480e-02
 * time: 8.119480848312378
    10    -6.189815e+01     2.668540e-02
 * time: 8.790104866027832
    11    -6.204364e+01     1.671566e-02
 * time: 9.428384780883789
    12    -6.210379e+01     1.498495e-02
 * time: 10.07347583770752
    13    -6.216531e+01     1.264215e-02
 * time: 10.711185932159424
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671063
    AtomicLocal         -18.8557700
    AtomicNonlocal      14.8522672
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336819

    total               -62.261666461198
